In [112]:
import matplotlib.image as img
from tqdm.notebook import tqdm
import numpy as np
from PIL import Image as im 
from PIL import ImageFont
from PIL import ImageDraw 
import matplotlib.pyplot as plt

Расчеты соотношения для размера (пиксель с микрометру)

In [94]:
px = 5000 / 45
35 * px

3888.888888888889

In [96]:
px = 5000 / 68
px * 44

3235.294117647059

Функции для расстояния между точками

In [98]:
import math

def distance(a, b):
    return math.hypot(*(v2 - v1 for v1, v2 in zip(a, b)))

def dis(x1, y1, x2, y2):
    return math.sqrt((x2-x1)**2 + (y2 - y1)**2)

Для установки уровней по цветам


In [120]:
first_dir = 'cells_3'
micro = 'micro'
cnt = '15'
file_name_base = 'prod_'

file_n = 1 # файл

file_name_main = file_name_base + str(file_n) + '.jpg'
file_name_dir = f'./{first_dir}/{micro}/{cnt}' + '/'
file_name_all = file_name_dir + file_name_main

image = img.imread(file_name_all)
image = np.array(image)

print('Image shape ==', image.shape)

file_name_2 = file_name_all[:1] + file_name_all[1:].split('.')[0] + '_1.' + file_name_all[1:].split('.')[1]

image_1 = img.imread(file_name_2)
image_1 = np.array(image_1)

print('Image mask shape ==', image_1.shape)

points_cell = []
for i in range(image_1.shape[0]):
    for j in range(image_1.shape[1]):
        green = np.array([0, 255, 0]) # цвет выделения маски
        h_green = distance(np.array(image_1[i][j]), green)
        if h_green < 120: # цифра отличия цвета маски от заданного (граница)
            points_cell.append([i, j])
            

print('Mask pixel size = ', len(points_cell))

# Проверка 1
image_1_copy = image_1.copy()
for i, j in points_cell:
    image_1_copy[i][j] = [255, 255, 255]
image_1_copy = im.fromarray(image_1_copy)
image_1_copy.show() # вывод вырезанной маски

    
points_d = []
points_t = []

for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        blue = np.array([0, 0, 255]) # цвет выделения точек диаметра
        green = np.array([0, 255, 0]) # цвет выделения границ клетки
        h_green = distance(np.array(image[i][j]), green)
        h_purple = distance(np.array(image[i][j]), blue)
        
        if h_green < 120: # цифра отличия цвета границ от заданного
            points_t.append([i, j])
            #print(h_green)
        elif h_purple < 70: # цифра отличия цвета точек диаметра от заданного
            points_d.append([i, j])


# Проверка 2
image_copy = image.copy()
for i, j in points_d:
    image_copy[i][j] = [255, 0, 0] # подсвечиваем найденные точки диаметра
for i, j in points_t:
    image_copy[i][j] = [0, 0, 255] # подсвечиваем найденные точки границ
image_copy = im.fromarray(image_copy)
image_copy.show() # вывод фото с подсвеченн

                

print('Points for target = ', len(points_t), ', Points for diametr = ', len(points_d))
    
nearest_points = []

for i in range(len(points_d)):
    for j in range(i+1, len(points_d)):
        if dis(points_d[i][0], points_d[i][1], points_d[j][0], points_d[j][1]) <= 10:
            nearest_points.append(i)
            
points_d = [points_d[i] for i in range(len(points_d)) if i not in nearest_points]
    
print(f'Diametr {len(points_d)} points = ', points_d)

Image shape == (944, 1517, 3)
Image mask shape == (944, 1517, 3)
Mask pixel size =  17614
Points for target =  922 , Points for diametr =  8
Diametr 2 points =  [[298, 1453], [304, 967]]


Расчет всего нужного циклом по картинкам

In [101]:
# ПРЕДУСТАНОВКИ

first_dir = 'cells_3'
micro = 'micro'
cnt = '15'
file_name_base = 'prod_'

imgs_cnt = 9


files = [i for i in range(1, imgs_cnt+1)]

for file_n in tqdm(files):

    print('================ FILE ', file_n, '===================')

    file_name_main = file_name_base + str(file_n) + '.jpg'
    file_name_dir = f'./{first_dir}/{micro}/{cnt}' + '/'
    file_name_all = file_name_dir + file_name_main

    try:
        image = img.imread(file_name_all)
        image = np.array(image)
    except:
        print('NO FILE', file_name_main, 'IN DIRECTORY', file_name_dir)
        print('\n\n\n')
        continue

    
    print('Image shape ==', image.shape)
    
    file_name_2 = file_name_all[:1] + file_name_all[1:].split('.')[0] + '_1.' + file_name_all[1:].split('.')[1]

    image_1 = img.imread(file_name_2)
    image_1 = np.array(image_1)

    print('Image mask shape ==', image_1.shape)
    
    points_cell = []
    for i in range(image_1.shape[0]):
        for j in range(image_1.shape[1]):
            green = np.array([0, 255, 0])
            h_green = distance(np.array(image_1[i][j]), green)
            if h_green < 120: # отсечка
                points_cell.append([i, j])
                
    
    print('Mask pixel size = ', len(points_cell))
        
    points_d = []
    points_t = []
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            blue = np.array([0, 0, 255])
            green = np.array([0, 255, 0])
            h_green = distance(np.array(image[i][j]), green)
            h_purple = distance(np.array(image[i][j]), blue)
            
            if h_green < 120: # отсечка
                points_t.append([i, j])
                #print(h_green)
            elif h_purple < 70: # отсечка
                points_d.append([i, j])
    
                    
    
    print('Points for target = ', len(points_t), ', Points for diametr = ', len(points_d))
        
    nearest_points = []
    
    for i in range(len(points_d)):
        for j in range(i+1, len(points_d)):
            if dis(points_d[i][0], points_d[i][1], points_d[j][0], points_d[j][1]) <= 10:
                nearest_points.append(i)
                
    points_d = [points_d[i] for i in range(len(points_d)) if i not in nearest_points]
        
    print(f'Diametr {len(points_d)} points = ', points_d)
    
    points_d_sort = []
    for p in range(0, len(points_d)):
        for p2 in range(p+1, len(points_d)):
            points_d_sort.append(dis(points_d[p][0], points_d[p][1], points_d[p2][0], points_d[p2][1]))
    
    if len(points_d_sort) > 2:
        diam = sum(sorted(points_d_sort)[0:2]) / 2
    else:
        diam = sum(points_d_sort) / len(points_d_sort)
    
    print('Total D = ', diam)
        
    um_for_pix = 3888.889 / diam
    print(f'µm for pixel == {um_for_pix}')
    
    diam_sf = []
    centers = []
    
    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)

    #print(points_t)
        
    print('Подсчет диаметра таргера')
    
    for a in points_t:
        for b in points_t:
            for p in [[x, y],
                        [x+1, y+1], [x+2, y+2],
                        [x-1, y-1], [x-2, y-2],
                        [x+1, y-1], [x-1, y+1],
                        [x-1, y], [x, y-1],
                        [x+1, y], [x, y+1]]:
                
                first_case = ((p[0] - a[0]) * (b[1] - a[1]) - (b[0] - a[0]) * (p[1] - a[1]) == 0)
                distance_case = (distance(a, b) > 10)
                points_case = (p[0] >= a[0] and p[0] <= b[0] and p[1] <= a[1] and p[1] >= b[1])
                    
                if first_case and distance_case and points_case:
                        
                    d = dis(a[0], a[1], b[0], b[1])
                    
                    if d > 0:
                        diam_sf.append(d)
                        centers.append([(a[0] + b[0]) / 2, (a[1] + b[1]) / 2])
                    if d < 5:
                        print('!!!! - ', a, b, p)
                            
    print(f'Диаметр таргета высчитан по {len(diam_sf)} точкам')
            
    r = int(np.mean(diam_sf) / 2)
    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)
    
    x_min = min([i[0] for i in points_t])
    y_min = min([i[1] for i in points_t])
    x_max = max([i[0] for i in points_t])
    y_max = max([i[1] for i in points_t])
    
    points_area = 0
    points_in_c = 0
        
    print('Подсчет округлости')
    
    for i in range(x_min, x_max + 1):
        for j in range(y_min, y_max + 1):
            if points_cell.count([i, j]) > 0:
                points_area += 1
                if (r ** 2) >= ((x - i) ** 2 + (y - j) ** 2):
                    points_in_c += 1
                    
    print(f'Диаметр посчитан на {len(diam_sf)} отрезках')
    print('Диаметр = ', round(np.mean(diam_sf) * um_for_pix / 1000, 3), 'mm')
    print('Диаметр = ', round(np.mean(diam_sf) * um_for_pix) , 'µm')
    print('Круглость = ', round(points_in_c / points_area * 100, 1), '%')
    
    result_txt = f'''
    Диаметр = {round(np.mean(diam_sf) * um_for_pix / 1000, 3)} mm
    Диаметр = {round(np.mean(diam_sf) * um_for_pix)} µm
    Круглость = {round(points_in_c / points_area * 100, 1)} %
    Отрезков диаметра таргета = {len(diam_sf)} штук
    Диаметр окружности в пикселях =  {diam}
    Точки диаметра: {points_d}
    µm for pixel == {um_for_pix}
    '''
        
    # Проверка
    
    image_copy = image.copy()
    
    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)
    
    for i in range(image_copy.shape[0]):
        for j in range(image_copy.shape[1]):
            cor_const = 3
            if i >= x-cor_const and i <= x+cor_const and j >= y-cor_const and j <= y+cor_const:
                image_copy[i][j] = [0, 0, 0]
    
    for i, j in points_t:
        image_copy[i][j] = [255, 0, 0]
        
    for i, j in points_d:
        image_copy[i][j] = [255, 0, 0]
        image_copy[i+1][j] = [255, 0, 0]
        image_copy[i][j+1] = [255, 0, 0]
        image_copy[i-1][j] = [255, 0, 0]
        image_copy[i][j-1] = [255, 0, 0]
        
        
    r = int(np.mean(diam_sf) / 2)
    x, y = np.mean(np.array(points_t), axis=0)
    x = int(x)
    y = int(y)
    
    for i in range(x - r, x + r + 1):
        for j in range(y - r, y + r + 1):
            r_2 = (r ** 2)
            circle = ((x - i) ** 2 + (y - j) ** 2)
            cor_const = 5
            if abs(r_2 - circle) <= cor_const:
                image_copy[i][j] = [0, 0, 255]
    
    k = im.fromarray(image_copy) 
    # Проверка
        
    print('Проверка прошла')
        
        
    x = max([i[0] for i in points_t]) + 25
    y = min([i[1] for i in points_t]) + 25
    color = [0, 36, 255]
    
    for i in range(y, int(y + 1000 / um_for_pix)):
        image[x][i] = color
        image[x+1][i] = color
        image[x+2][i] = color
        
    image[x-1][y] = color
    image[x-1][y] = color
    image[x-2][y] = color
    image[x-2][y] = color
    image[x+3][y] = color
    image[x+3][y] = color
    image[x+4][y] = color
    image[x+4][y] = color
    
    image[x-1][int(y + 1000 / um_for_pix)] = color
    image[x-1][int(y + 1000 / um_for_pix)] = color
    image[x-2][int(y + 1000 / um_for_pix)] = color
    image[x-2][int(y + 1000 / um_for_pix)] = color
    image[x+3][int(y + 1000 / um_for_pix)] = color
    image[x+3][int(y + 1000 / um_for_pix)] = color
    image[x+4][int(y + 1000 / um_for_pix)] = color
    image[x+4][int(y + 1000 / um_for_pix)] = color
        
    f = im.fromarray(image) 
        
    draw = ImageDraw.Draw(f)
    font = ImageFont.truetype("./arial.ttf", 22);
    draw.text((y, x + 10),"1000 µm = 1 mm",(0, 36, 255),font=font)
    
    print('Закончили, сохраняем')
        
    f.save(f'./{first_dir}/{micro}/{cnt}' + '/results/' + file_name_base + str(file_n) + '.jpg')
    k.save(f'./{first_dir}/{micro}/{cnt}' + '/results/' + file_name_base + str(file_n) + '_kontrol.jpg')
    file = open(f'./{first_dir}/{micro}/{cnt}' + '/results/' + file_name_base + str(file_n) + '.txt', "w")
    file.write(result_txt)
    file.close()
               
    print('Все ок, сохранил')
    print('\n\n\n')

  0%|          | 0/9 [00:00<?, ?it/s]

================ FILE  1 ===================
Image shape == (944, 1517, 3)
Image mask shape == (944, 1517, 3)
Mask pixel size =  17614
Points for target =  922 , Points for diametr =  8
Diametr 2 points =  [[298, 1453], [304, 967]]
Total D =  486.03703562588726
µm for pixel == 8.001219485243833
Подсчет диаметра таргера
Диаметр таргета высчитан по 347 точкам
Подсчет округлости
Диаметр посчитан на 347 отрезках
Диаметр =  1.365 mm
Диаметр =  1365 µm
Круглость =  86.1 %
Проверка прошла
Закончили, сохраняем
Все ок, сохранил




================ FILE  2 ===================
Image shape == (1339, 1525, 3)
Image mask shape == (1339, 1525, 3)
Mask pixel size =  24449
Points for target =  740 , Points for diametr =  9
Diametr 2 points =  [[516, 981], [522, 1485]]
Total D =  504.03571302041684
µm for pixel == 7.715502889063089
Подсчет диаметра таргера
Диаметр таргета высчитан по 192 точкам
Подсчет округлости
Диаметр посчитан на 192 отрезках
Диаметр =  1.384 mm
Диаметр =  1384 µm
Круглость =  90.7 